<h1> Demux-EM </h1>
<br>
Simple visual implementation

In [1]:
import h5py
import numpy as np
import pandas as pd
import pegasusio as io
from demuxEM.tools import *
import pegasus as pg


<h3> Get HTO Counts Matrix and UMI Matrix </h3>
Both matrices come from .rds objects and were transformed into .csv and hdf5 during the pre processing step

In [2]:
hto_mat = "/Users/mylenemarianagonzalesandre/Development/Data-cluster/Count/filtered_feature_bc_matrix"
umi_mat = "/Users/mylenemarianagonzalesandre/Development/Data-cluster/UMI/filtered_feature_bc_matrix.h5"

In [3]:
hto_data = io.read_input(hto_mat)

2022-09-15 20:17:12,110 - pegasusio.text_utils - INFO - Detected mtx file in 10x v3 format.
2022-09-15 20:17:12,351 - pegasusio.readwrite - INFO - mtx file '/Users/mylenemarianagonzalesandre/Development/Data-cluster/Count/filtered_feature_bc_matrix' is loaded.
2022-09-15 20:17:12,352 - pegasusio.readwrite - INFO - Function 'read_input' finished in 0.36s.


In [4]:
umi_data = pg.read_input(input_file=umi_mat)

2022-09-15 20:17:13,165 - pegasusio.readwrite - INFO - 10x file '/Users/mylenemarianagonzalesandre/Development/Data-cluster/UMI/filtered_feature_bc_matrix.h5' is loaded.
2022-09-15 20:17:13,165 - pegasusio.readwrite - INFO - Function 'read_input' finished in 0.81s.


In [5]:
umi_data

MultimodalData object with 1 UnimodalData: 'GRCh38-premrna-rna'
    It currently binds to UnimodalData object GRCh38-premrna-rna

UnimodalData object with n_obs x n_vars = 4889 x 33694
    Genome: GRCh38-premrna; Modality: rna
    It contains 1 matrix: 'X'
    It currently binds to matrix 'X' as X

    var: 'featureid'
    uns: 'genome', 'modality'

In [6]:
pg.qc_metrics(umi_data)
pg.identify_robust_genes(umi_data)
pg.qc_metrics(hto_data)

2022-09-15 20:17:13,537 - pegasus.tools.preprocessing - INFO - After filtration, 28243/33694 genes are kept. Among 28243 genes, 25981 genes are robust.
2022-09-15 20:17:13,538 - pegasus.tools.preprocessing - INFO - Function 'identify_robust_genes' finished in 0.27s.


In [7]:
pg.estimate_background_probs(hto_data)

2022-09-15 20:17:13,723 - demuxEM.tools.demuxEM - INFO - Background probability distribution is estimated.


In [8]:
pg.demultiplex(umi_data, hto_data,alpha=0.8, alpha_noise=0.10,min_signal=9.0)

2022-09-15 20:17:13,771 - demuxEM.tools.demuxEM - WARNING - Warning: 11 cells do not have ADTs, percentage = 0.22%.
2022-09-15 20:20:51,860 - demuxEM.tools.demuxEM - INFO - Demultiplexing is done.


In [9]:
hto_data.obs.head()

,n_genes,n_counts,passed_qc,counts,hto_type,rna_type
barcodekey,,,,,,
AAACCCAAGACCATGG,6,658,True,658,signal,background
AAACCCAAGACCTGGA,6,718,True,718,signal,background
AAACCCAAGAGAGCAA,6,722,True,722,signal,background
AAACCCAAGAGATGCC,6,741,True,741,signal,background
AAACCCAAGATGAACT,6,718,True,718,signal,background


In [10]:
umi_data.obs.head()

,n_genes,n_counts,passed_qc,demux_type,assignment
barcodekey,,,,,
AAACCCAAGGTGTGAC,5147,21181,True,doublet,"Hash451_TotalSeqA,Hash454_TotalSeqA"
AAACCCAGTGAGAGGG,3010,7919,True,doublet,"Hash453_TotalSeqA,Hash451_TotalSeqA"
AAACGAAAGAATCTAG,1137,1954,True,unknown,
AAACGAACACATATGC,892,1238,True,unknown,
AAACGAACACGACTAT,2840,10121,True,singlet,Hash452_TotalSeqA


In [11]:
type(hto_data.obs)

pandas.core.frame.DataFrame

In [12]:
umi_data.obs.to_csv("/Users/mylenemarianagonzalesandre/Development/Results/Trial 7/results_demuxem.csv")

In [13]:
umi_data.obs

,n_genes,n_counts,passed_qc,demux_type,assignment
barcodekey,,,,,
AAACCCAAGGTGTGAC,5147,21181,True,doublet,"Hash451_TotalSeqA,Hash454_TotalSeqA"
AAACCCAGTGAGAGGG,3010,7919,True,doublet,"Hash453_TotalSeqA,Hash451_TotalSeqA"
AAACGAAAGAATCTAG,1137,1954,True,unknown,
AAACGAACACATATGC,892,1238,True,unknown,
AAACGAACACGACTAT,2840,10121,True,singlet,Hash452_TotalSeqA
...,...,...,...,...,...
TTTGTTGAGTCCCAGC,481,632,True,doublet,"Hash453_TotalSeqA,Hash451_TotalSeqA,Hash456_To..."
TTTGTTGCAGCGCGTT,1817,3240,True,singlet,Hash451_TotalSeqA
TTTGTTGGTTATCTTC,7035,42242,True,doublet,"Hash453_TotalSeqA,Hash451_TotalSeqA,Hash452_To..."


In [14]:
hto_data.current_matrix

<bound method MultimodalData.current_matrix of MultimodalData object with 1 UnimodalData: 'unknown-rna'
    It currently binds to UnimodalData object unknown-rna

UnimodalData object with n_obs x n_vars = 90910 x 6
    Genome: unknown; Modality: rna
    It contains 1 matrix: 'X'
    It currently binds to matrix 'X' as X

    obs: 'n_genes', 'n_counts', 'passed_qc', 'counts', 'hto_type', 'rna_type'
    var: 'featureid', 'featuretype'
    uns: 'genome', 'modality', 'background_probs'>

In [15]:
hto_data.obs

,n_genes,n_counts,passed_qc,counts,hto_type,rna_type
barcodekey,,,,,,
AAACCCAAGACCATGG,6,658,True,658,signal,background
AAACCCAAGACCTGGA,6,718,True,718,signal,background
AAACCCAAGAGAGCAA,6,722,True,722,signal,background
AAACCCAAGAGATGCC,6,741,True,741,signal,background
AAACCCAAGATGAACT,6,718,True,718,signal,background
...,...,...,...,...,...,...
TTTGTTGTCGCTTAAG,6,760,True,760,signal,signal
TTTGTTGTCGTTGCCT,6,1054,True,1054,signal,background
TTTGTTGTCTACGGTA,6,695,True,695,signal,background


In [16]:
umi_data.current_matrix

<bound method MultimodalData.current_matrix of MultimodalData object with 1 UnimodalData: 'GRCh38-premrna-rna'
    It currently binds to UnimodalData object GRCh38-premrna-rna

UnimodalData object with n_obs x n_vars = 4889 x 28243
    Genome: GRCh38-premrna; Modality: rna
    It contains 1 matrix: 'X'
    It currently binds to matrix 'X' as X

    obs: 'n_genes', 'n_counts', 'passed_qc', 'demux_type', 'assignment'
    var: 'featureid', 'n_cells', 'percent_cells', 'robust', 'highly_variable_features'
    obsm: 'raw_probs'
    uns: 'genome', 'modality', 'background_probs'>